In [ ]:
import boto3
from datetime import date
import time
import uuid
from credentials import *
import unittest

## Connect to DynamoDB

Initially, we connect to our local database we started as "dynamodb" container:

In [ ]:
client = boto3.client('dynamodb', 
  endpoint_url='http://dynamo:8000', 
  region_name='eu-central-1',
  aws_access_key_id=aws_access_key_id,
  aws_secret_access_key=aws_secret_access_key
)
client.list_tables()

## Create Table

Let's setup the table with our attributes and indexes:

In [ ]:
def create_app_table():
    return client.create_table(
        TableName="twitter",
        KeySchema=[
            {"AttributeName": "PK", "KeyType": "HASH"},
            {"AttributeName": "SK", "KeyType": "RANGE"},
        ],
        AttributeDefinitions=[
            {"AttributeName": "PK", "AttributeType": "S"},
            {"AttributeName": "SK", "AttributeType": "S"},
        ],
        ProvisionedThroughput={"ReadCapacityUnits": 10, "WriteCapacityUnits": 10},
    )

In [ ]:
def delete_app_table():
    try:
        return client.delete_table(TableName="twitter")
    except:
        return False

In [ ]:
delete_app_table()
create_app_table()

## Helpers

To setup some other keys later on, we create a function to extract the id from a key:

In [ ]:
def parse_id_from_key(key):
    return key.split("#")[-1]

parse_id_from_key("CHAN#7")

## Users

We create and save our users:

In [ ]:
uuid_entry1 = str(uuid.uuid4())
user_martin = {
    'PK': {'S': f"USER#{uuid_entry1}"},
    'SK': {'S': f"USER#{uuid_entry1}"},
    'Name': {'S': 'Martin Marsal'},
}

uuid_entry2 = str(uuid.uuid4())
user_christian = {
    'PK': {'S': f"USER#{uuid_entry2}"},
    'SK': {'S': f"USER#{uuid_entry2}"},
    'Name': {'S': 'Christian Diegmann'},
}

uuid_entry3 = str(uuid.uuid4())
user_robin = {
    'PK': {'S': f"USER#{uuid_entry3}"},
    'SK': {'S': f"USER#{uuid_entry3}"},
    'Name': {'S': 'Robin Schüle'},
}

def save_user(user):
    return client.put_item(
        TableName="twitter",
        Item=user
    )

# AP6
def find_user(primaryKey, sortKey):
    item = client.get_item(
        TableName="twitter",
        Key={
          'PK': { 'S': primaryKey },
          'SK': { 'S': sortKey }
        }
    )
    return item['Item'] if 'Item' in item else False

In [ ]:
save_user(user_martin)
find_user(user_martin['PK']['S'], user_martin['SK']['S'])

In [ ]:
save_user(user_christian)
find_user(user_christian['PK']['S'], user_christian['SK']['S'])

In [ ]:
save_user(user_robin)
find_user(user_robin['PK']['S'], user_robin['SK']['S'])

In [ ]:
def add_to_followers(primaryKey, follower):
    response = client.put_item(
        TableName="twitter",
        Item={
            'PK': {'S': primaryKey},
            'SK': {'S': f"FOLLOWER#{parse_id_from_key(follower['PK']['S'])}"},
            'Name': {'S': follower['Name']['S']},
        },
        )
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.get_item(
            TableName='twitter',
            Key={
                'PK': {'S': primaryKey},
                'SK': {'S': f"FOLLOWER#{parse_id_from_key(follower['PK']['S'])}"},
            }
        )

In [ ]:
add_to_followers(user_martin['PK']['S'], user_christian)

In [ ]:
add_to_followers(user_christian['PK']['S'], user_robin)

In [ ]:
add_to_followers(user_robin['PK']['S'], user_martin)

## 1st access pattern: Post a tweet

In [ ]:
uuid_entry4 = str(uuid.uuid4())
timestamp = int(time.time())
tweet = {'id': {'S': f"{uuid_entry4}"}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
uuid_entry5 = str(uuid.uuid4())
timestamp = int(time.time())
tweet_2 = {'id': {'S': f"{uuid_entry5}"}, 'text': {'S': 'MongoDB ist super!'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
newMartin = find_user(user_martin['PK']['S'], user_martin['SK']['S'])

def post_tweet(primaryKey, tweet):
    client.put_item(
        TableName='twitter',
        Item={
            'PK': {'S': primaryKey},
            'SK': {'S': f"TWEET#{tweet['id']['S']}"},
            'id': {'S': tweet['id']['S']},
            'text': {'S': tweet['text']['S']},
            'likes': {'N': tweet['likes']['N']},
            'CreatedAt': { 'N': tweet['CreatedAt']['N']}
        },
    )
    item = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': { 'S': primaryKey },
            ':follower': { 'S': 'FOLLOWER#' }
        },
    )
    followers = item['Items']
    for follower in followers:
        response2 = client.put_item(
            TableName='twitter',
            Item={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweet['id']['S']}"},
                'tweetId': {'S': tweet['id']['S']},
                'text': {'S': tweet['text']['S']},
                'likes': {'N': tweet['likes']['N']},
                'CreatedAt': { 'N': tweet['CreatedAt']['N']}
            },
        )
    if response2['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': f"USER#{parse_id_from_key(user_christian['PK']['S'])}" },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )

In [ ]:
post_tweet(user_martin['PK']['S'], tweet)

In [ ]:
post_tweet(user_martin['PK']['S'], tweet_2)

In [ ]:
class TestPostTweet(unittest.TestCase):
    def test_post_tweet_enriched_timeline(self):
        christian_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(christian_timeline['Items'])
        self.assertEqual(timeline_length, 2)

    def test_post_tweet_empty_timeline(self):
        robin_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_robin['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(robin_timeline['Items'])
        self.assertEqual(timeline_length, 0)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

## 2nd access pattern: Write a reply

In [ ]:
uuid_entry6 = str(uuid.uuid4())
reply = {'id': {'S': f"{uuid_entry6}"}, 'text': {'S': 'Hallo zurück.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}, 'tweetId': {'S': f"{parse_id_from_key(tweet['id']['S'])}"},}

def post_reply(author, user, tweet, reply):
    client.put_item(
        TableName='twitter',
        Item={
            'PK': {'S': f"USER#{parse_id_from_key(author['PK']['S'])}"},
            'SK': {'S': f"TWEET#{tweet['id']['S']}#{reply['id']['S']}"},
            'id': {'S': reply['id']['S']},
            'text': {'S': reply['text']['S']},
            'likes': {'N': reply['likes']['N']},
            'CreatedAt': { 'N': reply['CreatedAt']['N']}
        },
    )
    item = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': { 'S': f"USER#{parse_id_from_key(author['PK']['S'])}" },
            ':follower': { 'S': 'FOLLOWER#' }
        },
    )
    followers = item['Items']
    for follower in followers:
        response2 = client.put_item(
            TableName='twitter',
            Item={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweet['id']['S']}#{reply['id']['S']}"},
                'id': {'S': reply['id']['S']},
                'text': {'S': reply['text']['S']},
                'likes': {'N': reply['likes']['N']},
                'CreatedAt': { 'N': reply['CreatedAt']['N']}
            },
        )
    if response2['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': f"USER#{parse_id_from_key(author['PK']['S'])}" },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )

In [ ]:
post_reply(user_martin, user_christian, tweet, reply)

In [ ]:
class TestPostReply(unittest.TestCase):
    def test_post_reply_enriched_replies(self):
        test_tweet = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': f"TIMELINE#{tweet['id']['S']}" }
            },
        )
        tweet_length = len(test_tweet['Items'])
        self.assertEqual(tweet_length, 2)
        
    def test_post_reply_empty_replies(self):
        test_tweet = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': f"TIMELINE#{tweet_2['id']['S']}" }
            },
        )
        tweet_length = len(test_tweet['Items'])
        self.assertEqual(tweet_length, 1)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostReply)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 3rd access pattern: Edit a tweet

In [ ]:
def edit_tweet(primaryKey, tweetId, new_text):

    response = client.update_item(
        TableName="twitter",
        Key={
            'PK': {'S': primaryKey},
            'SK': {'S': f"TWEET#{tweetId}"}
        },
        UpdateExpression='SET #text = :new_text',
        ExpressionAttributeNames={
            '#text': 'text'
        },
        ExpressionAttributeValues={
            ':new_text': {'S': new_text}
        }
    )


    followers_query = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':follower': {'S': 'FOLLOWER#'}
        },
    )

    followers = followers_query.get('Items', [])
    for follower in followers:
        client.update_item(
            TableName="twitter",
            Key={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweetId}"}
            },
            UpdateExpression='SET #text = :new_text',
            ExpressionAttributeNames={
                '#text': 'text'
            },
            ExpressionAttributeValues={
                ':new_text': {'S': new_text}
            }
        )

    return response

In [ ]:
new_tweet_text = "DynamoDB ist super"
edit_tweet(user_martin['PK']['S'], uuid_entry5, new_tweet_text)

In [ ]:
def get_user_tweets(primaryKey):
    response = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :tweet)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':tweet': {'S': 'TWEET#'}
        }
    )

    tweets = response.get('Items', [])
    return tweets

In [ ]:
user_martin_tweets = get_user_tweets(user_martin['PK']['S'])
print(user_martin_tweets)

In [ ]:
class TestEditTweet(unittest.TestCase):
    def test_edit_tweet__timeline(self):
        tweet_id_to_find = uuid_entry5  
        updated_text = "DynamoDB ist super"

        christian_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND SK = :sk',
            ExpressionAttributeValues={
                ':pk': {'S': f"USER#{parse_id_from_key(user_christian['PK']['S'])}"},
                ':sk': {'S': f"TIMELINE#{tweet_id_to_find}"}
            },
        )
        matching_items = christian_timeline.get('Items', [])

        self.assertTrue(
            any('text' in item and item['text']['S'] == updated_text for item in matching_items),
            f"Tweet with ID {tweet_id_to_find} not found in timeline or text not updated."
        )

    def test_edit_tweet_martin(self):
        tweet_id_to_find = uuid_entry5  
        updated_text = "DynamoDB ist super"  

        
        martin_tweets = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND SK = :sk',
            ExpressionAttributeValues={
                ':pk': {'S': f"USER#{parse_id_from_key(user_martin['PK']['S'])}"},
                ':sk': {'S': f"TWEET#{tweet_id_to_find}"}
            },
        )
        matching_items_martin = martin_tweets.get('Items', [])

        # Check if the updated text is found in user_martin's tweets
        self.assertTrue(
            any('text' in item and item['text']['S'] == updated_text for item in matching_items_martin),
            f"Tweet with ID {tweet_id_to_find} not found in Martin's tweets or text not updated."
        )


    def test_edit_tweet_empty_timeline(self):
        robin_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_robin['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(robin_timeline['Items'])
        self.assertEqual(timeline_length, 0)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestEditTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)


# 4th access pattern: Read a timeline

In [ ]:
def read_timeline(primaryKey):
    timeline_query = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':timeline': {'S': 'TIMELINE#'}
        },
    )
    return timeline_query.get('Items', [])

In [ ]:
user_christian_timeline = read_timeline(user_christian['PK']['S'])
print(user_christian_timeline)

In [ ]:
class TestReadTimeline(unittest.TestCase):
    def test_read_timeline(self):
        # Assuming user_christian has a populated timeline
        expected_texts = ["Moin, moin.", "Hallo zurück.", "DynamoDB ist super"]

        # Fetch the timeline of user_christian
        christian_timeline = read_timeline(user_christian['PK']['S'])

        # Extract text items from the timeline
        actual_texts = [item['text']['S'] for item in christian_timeline]

        # Check if the actual text items match the expected texts (order doesn't matter)
        self.assertCountEqual(actual_texts, expected_texts, "Timeline text items do not match the expected texts.")

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestReadTimeline)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 5th access pattern: Delete a Tweet

In [ ]:
def delete_tweet(primaryKey, tweet_id):
    # Löschen des Tweets aus der Tweet-Entität
    client.delete_item(
        TableName='twitter',
        Key={'PK': {'S': primaryKey}, 'SK': {'S': f"TWEET#{tweet_id}"}}
    )

    # Löschen des Tweets aus den Timelines aller Follower
    followers = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':msg': {'S': 'FOLLOWER#'}
        },
    )['Items']

    for follower in followers:
        client.delete_item(
            TableName='twitter',
            Key={'PK': {'S': follower['SK']['S']}, 'SK': {'S': f"TIMELINE#{tweet_id}"}}
        )


In [ ]:
def get_all_tweets():
    # Abrufen aller Tweets
    response = client.scan(
        TableName='twitter',
        FilterExpression='begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':msg': {'S': 'TWEET#'}
        }
    )

    tweets = response['Items']
    return tweets

In [ ]:
# Hier rufen wir alle Tweets ab, um sie vor dem Löschen anzuzeigen
all_tweets_before_deletion = get_all_tweets()
print(f'Alle Tweets vor der Löschung: {all_tweets_before_deletion}')

In [ ]:
print(f'USER: {uuid_entry1}')
print(f'UUID: {uuid_entry5}')

In [ ]:
# Tweet_2 bei User Martin löschen
delete_tweet(f'USER#{uuid_entry1}', uuid_entry5)

In [ ]:
# Hier rufen wir erneut alle Tweets ab, um die Änderungen anzuzeigen
all_tweets_after_deletion = get_all_tweets()
print(f'Alle Tweets nach der Löschung: {all_tweets_after_deletion}')

# 6th access pattern: Delete a User

In [ ]:
def delete_user(user_id):
    # Zuerst die Tweets des Benutzers abrufen
    user_tweets_response = client.query(
        TableName='twitter',
        KeyConditionExpression='#pk = :pk',
        ExpressionAttributeNames={'#pk': 'PK'},
        ExpressionAttributeValues={':pk': {'S': f'USER#{user_id}'}}
    )

    user_tweets = user_tweets_response.get('Items', [])

    # Dann jeden Tweet löschen
    for tweet in user_tweets:
        tweet_id = tweet.get('id', {}).get('S')  # Zugriff auf das tatsächlich vorhandene Feld 'id'

        if tweet_id:
            client.delete_item(
                TableName='twitter',
                Key={'PK': {'S': tweet['PK']['S']}, 'SK': {'S': tweet['SK']['S']}}
            )

            # Auch den Tweet aus den Timelines der Follower löschen
            followers_response = client.query(
                TableName='twitter',
                KeyConditionExpression='#pk = :pk',
                ExpressionAttributeNames={'#pk': 'PK'},
                ExpressionAttributeValues={':pk': {'S': tweet['PK']['S']}}
            )

            for follower in followers_response.get('Items', []):
                client.delete_item(
                    TableName='twitter',
                    Key={'PK': {'S': follower['SK']['S']}, 'SK': {'S': f'TIMELINE#{tweet_id}'}}
                )

    # Schließlich den Benutzer selbst löschen
    client.delete_item(
        TableName='twitter',
        Key={'PK': {'S': f'USER#{user_id}'}, 'SK': {'S': f'USER#{user_id}'}}
    )

In [ ]:
def get_all_users():
    response = client.scan(
        TableName='twitter',
        FilterExpression='begins_with(PK, :pk) AND begins_with(SK, :sk)',
        ExpressionAttributeValues={':pk': {'S': 'USER#'}, ':sk': {'S': 'USER#'}}
    )

    users = response['Items']
    return users

In [ ]:
# Alle Benutzer vor dem Löschen ausgeben
all_users_before_deletion = get_all_users()
print(f'Alle Benutzer vor dem Löschen: {all_users_before_deletion}')

In [ ]:
user_christian_timeline = read_timeline(user_christian['PK']['S'])
print(user_christian_timeline)

In [ ]:
delete_user(uuid_entry1)

In [ ]:
# Alle Benutzer nach dem Löschen ausgeben
all_users_after_deletion = get_all_users()
print(f'Alle Benutzer nach dem Löschen: {all_users_after_deletion}')

## Channel

We create our channel and functions to save, find and count users and messages in the channel:

In [ ]:
channel_town_hall = {
    'PK': { 'S': "CHAN#7" },
    'SK': { 'S': "CHAN#7" },
    'Name': { 'S': 'Town Hall' },
    'Desc': { 'S': 'General News' },
    'UserCount': { 'N': "0" },
    'MessageCount': { 'N': "0" }
}

def save_channel(channel):
    return client.put_item(
        TableName="chat",
        Item=channel
    )

# AP2
def find_channel(key):
    item = client.get_item(
        TableName="chat",
        Key={
          'PK': { 'S': key },
          'SK': { 'S': key }
        }
    )
    return item['Item'] if 'Item' in item else False

# AP4
def message_count_for_channel(channel):
    return int(channel['MessageCount']['N'])

# AP7
def user_count_for_channel(channel):
    return int(channel['UserCount']['N'])

save_channel(channel_town_hall)
channel_town_hall = find_channel(channel_town_hall['PK']['S'])
channel_town_hall

In [ ]:
message_count_for_channel(channel_town_hall)

In [ ]:
user_count_for_channel(channel_town_hall)

## User Join

When a user joins a channel, we create a new item and increment the user counter. Further, we setup queries to retreive the users for a channel and channels of a user:

In [ ]:
def join_channel(user, channel):
    # create userj record
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': "USERJ#" + parse_id_from_key(user['PK']['S']) },
            'Name': user['Name'],
            'JoinedAt': { 'S': str(date.today()) },
            'GSI1PK': user['PK'],
            'GSI1SK': channel['PK'],
            'ChanName': channel['Name']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD UserCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP5
def users_in_channel(key):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :userj)',
        ExpressionAttributeValues={
            ':pk': key,
            ':userj': { 'S': 'USERJ#' }
        }
    )
    return item['Items'] if 'Items' in item else []

# AP1
def channels_for_user(key):
    item = client.query(
        TableName="chat",
        IndexName="GSI1",
        KeyConditionExpression='GSI1PK = :pk AND begins_with(GSI1SK, :chan)',
        ExpressionAttributeValues={
            ':pk': key,
            ':chan': { 'S': 'CHAN#' }
        }
    )
    return item['Items'] if 'Items' in item else []

join_channel(user_alice, channel_town_hall)
join_channel(user_bob, channel_town_hall)
users_in_channel(channel_town_hall['PK'])

In [ ]:
channels_for_user(user_alice['PK'])

In [ ]:
find_channel(channel_town_hall['PK']['S'])

## Messages

When we send a message, we create a new item and increment the messages counter:

In [ ]:
def send_message(user, channel, text):
    timestamp = int(time.time())
    message_id = uuid.uuid4()
    
    # create message
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': f"MSG#{timestamp}#{message_id}" },
            'Msg': { 'S': text },
            'CreatedAt': { 'N': str(timestamp) },
            'UserName': user['Name'],
            'UserId': user['PK']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD MessageCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP3
def messages_in_channel(key, limit = 50):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':pk': key,
            ':msg': { 'S': 'MSG#' }
        },
        Limit = limit
    )
    return item['Items'] if 'Items' in item else []
    

send_message(user_bob, channel_town_hall, "Hey there!")
send_message(user_alice, channel_town_hall, "Hello Bob!")
send_message(user_bob, channel_town_hall, "How are you?")
messages_in_channel(channel_town_hall['PK'])

In [ ]:
find_channel(channel_town_hall['PK']['S'])